# Ensemble Tradicional - Metodos Clasicos de Ensemble

Implementacion academica de metodos de ensemble tradicionales usando los mejores algoritmos identificados en notebooks anteriores (03-06). Este notebook se enfoca en demostrar la metodologia clasica de ensemble con evaluacion rigurosa.

## Objetivos Academicos
1. Implementar metodos de ensemble clasicos: Voting, Bagging, Stacking
2. Usar los mejores algoritmos de cada categoria (notebooks 03-06) 
3. Evaluar sistematicamente cada tipo de ensemble
4. Comparar rendimiento ensemble vs algoritmos individuales
5. Proporcionar base teorica para ensembles avanzados

## Enfoque Metodologico
- **Voting Ensembles**: Hard/Soft voting con mejores algoritmos
- **Bagging**: Bootstrap aggregating con diferentes estimadores base  
- **Stacking**: Meta-learning con validacion cruzada
- **Evaluacion rigurosa**: Cross-validation y metricas multiples

Los resultados esperados son F1-scores entre 0.65-0.72 que demuestran la efectividad de combinar algoritmos complementarios para deteccion de fake news.

In [ ]:
# Configuracion de entorno y dependencias
import sys
import subprocess

# Detectar si estamos en Colab
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("Ejecutando en Google Colab")
    # Instalar dependencias para Colab
    !pip install plotly scikit-learn xgboost lightgbm
    # Montar Google Drive si es necesario
    from google.colab import drive
    # drive.mount('/content/drive')
else:
    print("Ejecutando en entorno local")
    # Verificar dependencias locales
    packages = ['plotly', 'scikit-learn', 'xgboost', 'lightgbm']
    for package in packages:
        try:
            __import__(package.replace('-', '_'))
        except ImportError:
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])

print(f"Entorno configurado: {'Google Colab' if IN_COLAB else 'Local'}")



In [ ]:
# Importar librerias necesarias para ensemble
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Machine learning y ensemble
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, f1_score, accuracy_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer

# Algoritmos base
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis

# Metodos de ensemble
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier, StackingClassifier
import xgboost as xgb
import lightgbm as lgb
import catboost as cb

# Procesamiento de texto
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Configuracion
np.random.seed(42)
if IN_COLAB:
    plt.style.use('default')
else:
    plt.rcParams['figure.figsize'] = (10, 6)

nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)


## Carga y Preparacion de Datos

Cargo el dataset Truth Seeker y preparo los datos siguiendo las mejores practicas para evitar data leakage.

In [3]:
# Cargar los datos procesados del notebook 02_limpieza_datos
if IN_COLAB:
    # Para Colab - ajustar segun donde subas los datos
    df_features = pd.read_csv('dataset_features_processed.csv')
    df_text = pd.read_csv('text_data_for_nlp.csv')
else:
    # Para ejecucion local - usar datos procesados
    df_features = pd.read_csv('../processed_data/dataset_features_processed.csv')
    df_text = pd.read_csv('../processed_data/text_data_for_nlp.csv')

print(f"Dataset de features cargado: {df_features.shape[0]} filas y {df_features.shape[1]} columnas.")
print(f"Dataset de texto cargado: {df_text.shape[0]} filas y {df_text.shape[1]} columnas.")

# Combinar datos de features y texto por indice
df = pd.concat([df_features, df_text], axis=1)
print(f"Dataset combinado: {df.shape[0]} filas y {df.shape[1]} columnas.")

print("\nInformacion del dataset combinado:")
print(df.info())

# Verificar columnas disponibles
target_columns = [col for col in df.columns if 'target' in col.lower()]
text_columns = [col for col in df.columns if 'statement' in col.lower()]

print(f"\nColumnas target disponibles: {target_columns}")
print(f"Columnas texto disponibles: {text_columns}")

# Seleccionar columna target apropiada
if 'BinaryNumTarget' in df.columns:
    target_col = 'BinaryNumTarget'
else:
    target_col = target_columns[0] if target_columns else None

# Seleccionar columna texto
text_col = 'statement' if 'statement' in df.columns else (text_columns[0] if text_columns else None)

print(f"\nUsando columna target: {target_col}")
print(f"Usando columna texto: {text_col}")

if target_col:
    print(f"\nDistribucion de etiquetas:")
    print(df[target_col].value_counts())
    print("\nPorcentajes:")
    print(df[target_col].value_counts(normalize=True) * 100)

Dataset de features cargado: 134198 filas y 58 columnas.
Dataset de texto cargado: 134198 filas y 2 columnas.
Dataset combinado: 134198 filas y 60 columnas.

Informacion del dataset combinado:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134198 entries, 0 to 134197
Data columns (total 60 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   BinaryNumTarget         134198 non-null  float64
 1   followers_count         134198 non-null  float64
 2   friends_count           134198 non-null  float64
 3   favourites_count        134198 non-null  float64
 4   statuses_count          134198 non-null  float64
 5   listed_count            134198 non-null  float64
 6   BotScore                134198 non-null  float64
 7   BotScoreBinary          134198 non-null  float64
 8   cred                    134198 non-null  float64
 9   normalize_influence     134198 non-null  float64
 10  mentions                134198 non-null  fl

In [4]:
print(f"Total de filas: {len(df)}")

# Correccion critica de data leakage: eliminar duplicados por statement
# PROBLEMA: El dataset original contiene el mismo statement repetido multiples veces
# con diferentes tweets, creando data leakage masivo cuando se usa texto en ensemble
# SOLUCION: Eliminar duplicados basados en statement antes de train/test split
# JUSTIFICACION: Cada statement unico debe aparecer solo una vez para evitar que
# el mismo contenido aparezca en entrenamiento y prueba simultaneamente
if text_col and text_col in df.columns:
    initial_size = len(df)
    df_clean = df.drop_duplicates(subset=[text_col], keep='first')
    final_size = len(df_clean)
    duplicates_removed = initial_size - final_size
    print(f"Data leakage corregido: eliminados {duplicates_removed:,} duplicados de statements")
    print(f"Dataset reducido de {initial_size:,} a {final_size:,} muestras unicas")
else:
    df_clean = df.copy()
    print("No se encontro columna de texto para corregir duplicados")

# Verificar que no tengamos data leakage - el target debe ser BinaryNumTarget
if target_col:
    print(f"\nDistribucion de etiquetas en datos procesados:")
    print(df_clean[target_col].value_counts())
    print("\nPorcentajes:")
    print(df_clean[target_col].value_counts(normalize=True) * 100)

print("\nDatos procesados listos para ensemble - data leakage corregido.")

Total de filas: 134198
Data leakage corregido: eliminados 133,140 duplicados de statements
Dataset reducido de 134,198 a 1,058 muestras unicas

Distribucion de etiquetas en datos procesados:
BinaryNumTarget
1.0    579
0.0    479
Name: count, dtype: int64

Porcentajes:
BinaryNumTarget
1.0    54.725898
0.0    45.274102
Name: proportion, dtype: float64

Datos procesados listos para ensemble - data leakage corregido.


In [5]:
# Visualizar la distribucion de los datos procesados
if target_col:
    fig = make_subplots(rows=1, cols=1, specs=[[{'type':'domain'}]],
                        subplot_titles=['Distribucion de Clases - Datos Procesados'])

    # Dataset procesado
    fig.add_trace(go.Pie(labels=df_clean[target_col].value_counts().index,
                         values=df_clean[target_col].value_counts().values,
                         name="Procesado"))

    fig.update_traces(hole=.4, hoverinfo="label+percent+name")
    fig.update_layout(title="Distribucion de Clases en Datos Procesados")
    fig.show()

    print("Los datos procesados mantienen una distribucion balanceada para el ensemble.")

Los datos procesados mantienen una distribucion balanceada para el ensemble.


## Preparacion de Features para Ensemble

En lugar de usar solo texto, ahora tenemos tanto features numericas procesadas como texto. Combino ambos enfoques para un ensemble mas robusto.

In [6]:
# Preparar diferentes tipos de features para ensemble
print("Preparando features para ensemble robusto...")

# 1. Features numericas ya procesadas (escaladas) del notebook 02
numeric_features = [col for col in df_clean.columns 
                   if col != target_col and col not in ['statement', 'tweet']]
X_numeric = df_clean[numeric_features].values
print(f"Features numericas: {len(numeric_features)} columnas")
print(f"Forma de features numericas: {X_numeric.shape}")

# 2. Preprocesamiento de texto si esta disponible
if text_col and text_col in df_clean.columns:
    print(f"\nPreprocesando texto de la columna: {text_col}")
    
    def preprocess_text(text):
        """Preprocesa texto para algoritmos de ensemble"""
        if pd.isna(text):
            return ""
        
        # Convertir a string y minusculas
        text = str(text).lower()
        
        # Eliminar caracteres especiales manteniendo espacios y puntuacion basica
        text = re.sub(r'[^a-zA-Z0-9\s\.,;:!?]', '', text)
        
        # Eliminar espacios multiples
        text = re.sub(r'\s+', ' ', text)
        
        # Eliminar espacios al inicio y final
        text = text.strip()
        
        return text
    
    df_clean['statement_processed'] = df_clean[text_col].apply(preprocess_text)
    
    # Eliminar textos vacios si existen
    initial_rows = len(df_clean)
    df_clean = df_clean[df_clean['statement_processed'].str.len() > 0].copy()
    final_rows = len(df_clean)
    
    if initial_rows != final_rows:
        print(f"Eliminadas {initial_rows - final_rows} filas con texto vacio.")
        # Actualizar X_numeric para mantener consistencia
        X_numeric = df_clean[numeric_features].values
    else:
        print("No se encontraron textos vacios.")
    
    X_text = df_clean['statement_processed'].values
    print(f"Features de texto preparadas: {len(X_text)} muestras")
    
    # Estadisticas de texto
    df_clean['text_length'] = df_clean['statement_processed'].str.len()
    df_clean['word_count'] = df_clean['statement_processed'].str.split().str.len()
    
    print("\nEstadisticas de texto:")
    print(df_clean[['text_length', 'word_count']].describe())
    
else:
    print("No hay columna de texto disponible")
    X_text = None

# Target
y = df_clean[target_col].values
print(f"\nTarget preparado: {len(y)} muestras")
print(f"Distribucion: {np.bincount(y.astype(int))}")

print(f"\nDatos finales para ensemble:")
print(f"- Features numericas: {X_numeric.shape}")
print(f"- Features texto: {'Disponible' if X_text is not None else 'No disponible'}")
print(f"- Target: {len(y)} muestras")

Preparando features para ensemble robusto...
Features numericas: 57 columnas
Forma de features numericas: (1058, 57)

Preprocesando texto de la columna: statement
No se encontraron textos vacios.
Features de texto preparadas: 1058 muestras

Estadisticas de texto:
       text_length   word_count
count  1058.000000  1058.000000
mean     92.419660    15.645558
std      39.313211     6.749756
min      21.000000     3.000000
25%      65.000000    11.000000
50%      85.000000    14.000000
75%     114.000000    19.000000
max     292.000000    48.000000

Target preparado: 1058 muestras
Distribucion: [479 579]

Datos finales para ensemble:
- Features numericas: (1058, 57)
- Features texto: Disponible
- Target: 1058 muestras


In [7]:
# Definir algoritmos base optimizados para ensemble
def get_base_classifiers():
    """Retorna mejores algoritmos de cada notebook (03-06)"""
    
    # Mejores tradicionales del notebook 03
    # Random Forest: 0.6749, Extra Trees: 0.6428, Logistic: 0.6411
    traditional_best = {
        'random_forest': RandomForestClassifier(n_estimators=100, random_state=42, max_depth=15),
        'extra_trees': ExtraTreesClassifier(n_estimators=100, random_state=42, max_depth=15),
        'logistic_regression': LogisticRegression(random_state=42, max_iter=1000, C=1.0)
    }
    
    # Mejores lineales del notebook 04  
    # Ridge CV: 0.6554, LogReg Elastic: 0.6552, Ridge Basic: 0.6547
    linear_best = {
        'ridge_classifier': RidgeClassifier(random_state=42, alpha=1.0),
        'logistic_elastic': LogisticRegression(random_state=42, penalty='elasticnet', 
                                              C=1.0, l1_ratio=0.5, solver='saga', max_iter=1000)
    }
    
    # Mejores otros algoritmos del notebook 06
    # Gaussian NB: 0.6822, KNN_15: 0.6665, LDA: 0.6547, SVM Linear: 0.6552
    other_best = {
        'gaussian_nb': GaussianNB(var_smoothing=1e-08),
        'knn_best': KNeighborsClassifier(n_neighbors=15, weights='distance'),
        'lda': LinearDiscriminantAnalysis(),
        'svm_linear': SVC(kernel='linear', probability=True, random_state=42, C=1.0)
    }
    
    # Combinar todos los mejores algoritmos
    base_classifiers = {}
    base_classifiers.update(traditional_best)
    base_classifiers.update(linear_best)
    base_classifiers.update(other_best)
    
    return base_classifiers

# Definir algoritmos de boosting del notebook 05
def get_boosting_classifiers():
    """Retorna mejores algoritmos de boosting del notebook 05"""
    
    # Mejores boosting del notebook 05
    # XGBoost Aggressive: 0.7150, LightGBM Balanced: 0.6991, HistGradient: 0.6959, CatBoost: 0.6856
    boosting_classifiers = {
        'xgboost_aggressive': xgb.XGBClassifier(
            n_estimators=200, learning_rate=0.15, max_depth=6, 
            subsample=0.9, colsample_bytree=0.9, gamma=0.1,
            random_state=42, eval_metric='logloss'
        ),
        'lightgbm_balanced': lgb.LGBMClassifier(
            n_estimators=150, learning_rate=0.08, max_depth=6,
            colsample_bytree=0.9, subsample=0.9, reg_alpha=0.1, reg_lambda=0.1,
            random_state=42, verbose=-1
        ),
        'hist_gradient_boost': GradientBoostingClassifier(
            n_estimators=100, learning_rate=0.1, max_depth=6,
            random_state=42, subsample=0.8
        )
    }
    
    # Agregar CatBoost si esta disponible
    try:
        boosting_classifiers['catboost_optimized'] = cb.CatBoostClassifier(
            iterations=150, learning_rate=0.08, depth=6, l2_leaf_reg=1,
            random_seed=42, verbose=False, bootstrap_type='Bayesian'
        )
    except:
        print("CatBoost no disponible")

    return boosting_classifiers

# Obtener los clasificadores
base_classifiers = get_base_classifiers()
boosting_classifiers = get_boosting_classifiers()

print(f"Algoritmos base definidos: {len(base_classifiers)}")
print("Mejores algoritmos por notebook:")
for name in base_classifiers.keys():
    print(f"  - {name}")

print(f"\nAlgoritmos de boosting definidos: {len(boosting_classifiers)}")
print("Mejores boosting del notebook 05:")
for name in boosting_classifiers.keys():
    print(f"  - {name}")

print(f"\nTotal algoritmos: {len(base_classifiers) + len(boosting_classifiers)}")

Algoritmos base definidos: 9
Mejores algoritmos por notebook:
  - random_forest
  - extra_trees
  - logistic_regression
  - ridge_classifier
  - logistic_elastic
  - gaussian_nb
  - knn_best
  - lda
  - svm_linear

Algoritmos de boosting definidos: 4
Mejores boosting del notebook 05:
  - xgboost_aggressive
  - lightgbm_balanced
  - hist_gradient_boost
  - catboost_optimized

Total algoritmos: 13


In [8]:
# Codificar las etiquetas si es necesario
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
print(f"Etiquetas codificadas: {label_encoder.classes_} -> {np.unique(y_encoded)}")

# Division en train/test estratificada
X_num_train, X_num_test, y_train, y_test = train_test_split(
    X_numeric, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

print(f"\nDivision de datos:")
print(f"Entrenamiento: {len(X_num_train)} muestras")
print(f"Prueba: {len(X_num_test)} muestras")
print(f"Distribucion entrenamiento: {np.bincount(y_train)}")
print(f"Distribucion prueba: {np.bincount(y_test)}")

# Dividir texto si esta disponible
if X_text is not None:
    X_text_train, X_text_test = train_test_split(
        X_text, test_size=0.2, random_state=42, stratify=y_encoded
    )
    
    # Vectorizacion TF-IDF del texto
    print("\nAplicando vectorizacion TF-IDF al texto...")
    tfidf_vectorizer = TfidfVectorizer(
        max_features=5000,  # Limitar features para evitar overfitting
        stop_words='english',
        ngram_range=(1, 2),  # Unigrams y bigrams
        min_df=2,
        max_df=0.95
    )
    
    X_text_tfidf_train = tfidf_vectorizer.fit_transform(X_text_train)
    X_text_tfidf_test = tfidf_vectorizer.transform(X_text_test)
    
    print(f"Features TF-IDF: {X_text_tfidf_train.shape[1]}")
    
    # Combinar features numericas y TF-IDF
    from scipy.sparse import hstack
    X_combined_train = hstack([X_num_train, X_text_tfidf_train])
    X_combined_test = hstack([X_num_test, X_text_tfidf_test])
    
    print(f"Features combinadas: {X_combined_train.shape[1]} ({X_num_train.shape[1]} num + {X_text_tfidf_train.shape[1]} texto)")
    
    # Para algoritmos que requieren arrays densos
    X_combined_dense_train = X_combined_train.toarray()
    X_combined_dense_test = X_combined_test.toarray()
    
    # Usar solo features numericas para algunos algoritmos
    X_train_tfidf = X_combined_train  # Para compatibilidad con el codigo original
    X_test_tfidf = X_combined_test
    X_train_dense = X_combined_dense_train
    X_test_dense = X_combined_dense_test
    
else:
    # Solo features numericas
    X_train_tfidf = X_num_train
    X_test_tfidf = X_num_test
    X_train_dense = X_num_train
    X_test_dense = X_num_test
    print("Usando solo features numericas procesadas")

print(f"\nDatos preparados para ensemble:")
print(f"Shape entrenamiento: {X_train_tfidf.shape}")
print(f"Shape prueba: {X_test_tfidf.shape}")
print("Listo para evaluar algoritmos de ensemble.")

Etiquetas codificadas: [0. 1.] -> [0 1]

Division de datos:
Entrenamiento: 846 muestras
Prueba: 212 muestras
Distribucion entrenamiento: [383 463]
Distribucion prueba: [ 96 116]

Aplicando vectorizacion TF-IDF al texto...
Features TF-IDF: 1450
Features combinadas: 1507 (57 num + 1450 texto)

Datos preparados para ensemble:
Shape entrenamiento: (846, 1507)
Shape prueba: (212, 1507)
Listo para evaluar algoritmos de ensemble.


## Algoritmos Base para Ensemble

Defino y configuro todos los algoritmos base que usare en los diferentes metodos de ensemble.

In [9]:
# Definir algoritmos base optimizados para ensemble
def get_base_classifiers():
    """Retorna diccionario con algoritmos base configurados"""
    
    base_classifiers = {
        # Modelos lineales
        'logistic_regression': LogisticRegression(random_state=42, max_iter=1000, C=1.0),
        'ridge_classifier': RidgeClassifier(random_state=42, alpha=1.0),
        
        # Modelos basados en arboles
        'decision_tree': DecisionTreeClassifier(random_state=42, max_depth=10, min_samples_split=10),
        'random_forest': RandomForestClassifier(n_estimators=100, random_state=42, max_depth=15),
        'extra_trees': ExtraTreesClassifier(n_estimators=100, random_state=42, max_depth=15),
        
        # SVM
        'svm_linear': SVC(kernel='linear', probability=True, random_state=42, C=1.0),
        'svm_rbf': SVC(kernel='rbf', probability=True, random_state=42, C=1.0),
        
        # Neighbors
        'knn': KNeighborsClassifier(n_neighbors=5),
        
        # Naive Bayes
        'multinomial_nb': MultinomialNB(alpha=1.0),
        'gaussian_nb': GaussianNB(),
        
        # Analisis discriminante
        'lda': LinearDiscriminantAnalysis(),
        'qda': QuadraticDiscriminantAnalysis()
    }
    
    return base_classifiers

# Definir algoritmos de boosting
def get_boosting_classifiers():
    """Retorna algoritmos de boosting optimizados"""
    
    boosting_classifiers = {
        'ada_boost': AdaBoostClassifier(n_estimators=100, random_state=42, learning_rate=1.0),
        'gradient_boost': GradientBoostingClassifier(n_estimators=100, random_state=42, learning_rate=0.1),
        'xgboost': xgb.XGBClassifier(n_estimators=100, random_state=42, learning_rate=0.1, eval_metric='logloss'),
        'lightgbm': lgb.LGBMClassifier(n_estimators=100, random_state=42, learning_rate=0.1, verbose=-1),
        'catboost': cb.CatBoostClassifier(iterations=100, random_state=42, learning_rate=0.1, verbose=False)
    }

    return boosting_classifiers

# Obtener los clasificadores
base_classifiers = get_base_classifiers()
boosting_classifiers = get_boosting_classifiers()

print(f"Algoritmos base definidos: {len(base_classifiers)}")
print("Lista de algoritmos base:")
for i, name in enumerate(base_classifiers.keys(), 1):
    print(f"  {i:2d}. {name}")

print(f"\nAlgoritmos de boosting definidos: {len(boosting_classifiers)}")
print("Lista de algoritmos de boosting:")
for i, name in enumerate(boosting_classifiers.keys(), 1):
    print(f"  {i:2d}. {name}")

Algoritmos base definidos: 12
Lista de algoritmos base:
   1. logistic_regression
   2. ridge_classifier
   3. decision_tree
   4. random_forest
   5. extra_trees
   6. svm_linear
   7. svm_rbf
   8. knn
   9. multinomial_nb
  10. gaussian_nb
  11. lda
  12. qda

Algoritmos de boosting definidos: 5
Lista de algoritmos de boosting:
   1. ada_boost
   2. gradient_boost
   3. xgboost
   4. lightgbm
   5. catboost


## Evaluacion Individual de Algoritmos Base

Evaluo cada algoritmo base individualmente para entender su rendimiento antes de combinarlos en ensembles.

In [10]:
# Funcion para evaluar un clasificador
def evaluate_classifier(classifier, X_train, X_test, y_train, y_test, name, use_dense=False):
    """Evalua un clasificador y retorna metricas"""
    
    # Seleccionar el tipo de datos segun el algoritmo
    if use_dense:
        X_tr = X_train_dense if hasattr(X_train, 'toarray') else X_train
        X_te = X_test_dense if hasattr(X_test, 'toarray') else X_test
    else:
        X_tr = X_train
        X_te = X_test
    
    try:
        # Entrenar el modelo
        classifier.fit(X_tr, y_train)
        
        # Generar predicciones
        y_pred = classifier.predict(X_te)
        
        # Predicciones probabilisticas si estan disponibles
        if hasattr(classifier, 'predict_proba'):
            y_pred_proba = classifier.predict_proba(X_te)[:, 1]
        elif hasattr(classifier, 'decision_function'):
            y_pred_proba = classifier.decision_function(X_te)
        else:
            y_pred_proba = None
        
        # Calcular metricas
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        
        if y_pred_proba is not None:
            try:
                roc_auc = roc_auc_score(y_test, y_pred_proba)
            except:
                roc_auc = 0.0
        else:
            roc_auc = 0.0
        
        return {
            'name': name,
            'accuracy': accuracy,
            'f1_score': f1,  # Correccion: usar f1_score consistente
            'roc_auc': roc_auc,
            'success': True,
            'error': None
        }
        
    except Exception as e:
        return {
            'name': name,
            'accuracy': 0.0,
            'f1_score': 0.0,
            'roc_auc': 0.0,
            'success': False,
            'error': str(e)
        }

# Evaluar todos los algoritmos base
print("Evaluando algoritmos base individualmente...")

base_results = []
# Excluir MultinomialNB porque no maneja valores negativos en features numericas escaladas
algorithms_need_dense = ['gaussian_nb', 'lda', 'qda']
algorithms_to_skip = ['multinomial_nb']  # Saltar MultinomialNB

for name, classifier in base_classifiers.items():
    if name in algorithms_to_skip:
        print(f"Saltando {name} (no compatible con features numericas negativas)")
        continue
        
    print(f"Evaluando {name}...", end=" ")
    
    # Determinar si necesita arrays densos
    use_dense = name in algorithms_need_dense
    
    result = evaluate_classifier(classifier, X_train_tfidf, X_test_tfidf, 
                               y_train, y_test, name, use_dense)
    base_results.append(result)
    
    if result['success']:
        print(f"F1: {result['f1_score']:.4f}, AUC: {result['roc_auc']:.4f}")
    else:
        print(f"ERROR: {result['error'][:50]}...")

print("\n" + "=" * 60)
print("Evaluacion de algoritmos base completada.")

Evaluando algoritmos base individualmente...
Evaluando logistic_regression... F1: 0.8155, AUC: 0.8636
Evaluando ridge_classifier... F1: 0.8261, AUC: 0.8869
Evaluando decision_tree... F1: 0.7737, AUC: 0.7393
Evaluando random_forest... F1: 0.8080, AUC: 0.8484
Evaluando extra_trees... F1: 0.8367, AUC: 0.8909
Evaluando svm_linear... F1: 0.8018, AUC: 0.8791
Evaluando svm_rbf... F1: 0.7073, AUC: 0.6658
Evaluando knn... F1: 0.6805, AUC: 0.6675
Saltando multinomial_nb (no compatible con features numericas negativas)
Evaluando gaussian_nb... F1: 0.8000, AUC: 0.8228
Evaluando lda... F1: 0.6379, AUC: 0.6541
Evaluando qda... F1: 0.5191, AUC: 0.4608

Evaluacion de algoritmos base completada.


In [11]:
# Evaluar algoritmos de boosting
print("Evaluando algoritmos de boosting...")

boosting_results = []

for name, classifier in boosting_classifiers.items():
    print(f"Evaluando {name}...", end=" ")
    
    # Los algoritmos de boosting generalmente manejan matrices sparse
    result = evaluate_classifier(classifier, X_train_tfidf, X_test_tfidf, 
                               y_train, y_test, name, use_dense=False)
    boosting_results.append(result)
    
    if result['success']:
        print(f"F1: {result['f1_score']:.4f}, AUC: {result['roc_auc']:.4f}")
    else:
        print(f"ERROR: {result['error'][:50]}...")

print("\n" + "=" * 60)
print("Evaluacion de algoritmos de boosting completada.")

Evaluando algoritmos de boosting...
Evaluando ada_boost... F1: 0.7801, AUC: 0.8216
Evaluando gradient_boost... F1: 0.8133, AUC: 0.8690
Evaluando xgboost... F1: 0.8051, AUC: 0.8570
Evaluando lightgbm... F1: 0.8117, AUC: 0.8491
Evaluando catboost... F1: 0.8500, AUC: 0.8744

Evaluacion de algoritmos de boosting completada.


In [12]:
# Crear DataFrame con todos los resultados individuales
all_individual_results = base_results + boosting_results
successful_results = [r for r in all_individual_results if r['success']]

individual_df = pd.DataFrame(successful_results)
individual_df = individual_df.sort_values('f1_score', ascending=False)

print("Resultados de algoritmos individuales (ordenados por F1-Score):")
print(individual_df[['name', 'accuracy', 'f1_score', 'roc_auc']].round(4))

# Identificar los mejores algoritmos para ensemble
top_5_algorithms = individual_df.head(5)['name'].tolist()
print(f"\nTop 5 algoritmos para usar en ensemble:")
for i, alg in enumerate(top_5_algorithms, 1):
    f1_score = individual_df[individual_df['name'] == alg]['f1_score'].iloc[0]
    print(f"  {i}. {alg} (F1: {f1_score:.4f})")

Resultados de algoritmos individuales (ordenados por F1-Score):
                   name  accuracy  f1_score  roc_auc
15             catboost    0.8302    0.8500   0.8744
4           extra_trees    0.8066    0.8367   0.8909
1      ridge_classifier    0.8113    0.8261   0.8869
0   logistic_regression    0.7972    0.8155   0.8636
12       gradient_boost    0.7877    0.8133   0.8690
14             lightgbm    0.7877    0.8117   0.8491
3         random_forest    0.7736    0.8080   0.8484
13              xgboost    0.7830    0.8051   0.8570
5            svm_linear    0.7877    0.8018   0.8791
8           gaussian_nb    0.7877    0.8000   0.8228
11            ada_boost    0.7500    0.7801   0.8216
2         decision_tree    0.7406    0.7737   0.7393
6               svm_rbf    0.5472    0.7073   0.6658
7                   knn    0.6368    0.6805   0.6675
9                   lda    0.6038    0.6379   0.6541
10                  qda    0.4670    0.5191   0.4608

Top 5 algoritmos para usar en ense

In [13]:
# Visualizar el rendimiento de los algoritmos individuales
fig = make_subplots(rows=1, cols=2, 
                    subplot_titles=['F1-Score por Algoritmo', 'ROC-AUC por Algoritmo'])

# F1-Score
fig.add_trace(go.Bar(
    x=individual_df['name'],
    y=individual_df['f1_score'],
    name='F1-Score',
    marker_color='lightblue'
), row=1, col=1)

# ROC-AUC
fig.add_trace(go.Bar(
    x=individual_df['name'],
    y=individual_df['roc_auc'],
    name='ROC-AUC',
    marker_color='lightcoral'
), row=1, col=2)

fig.update_layout(
    title="Rendimiento Individual de Algoritmos Base",
    height=500,
    showlegend=False
)

fig.update_xaxes(tickangle=45)
fig.show()

print("La visualizacion muestra el rendimiento individual de cada algoritmo antes del ensemble.")

La visualizacion muestra el rendimiento individual de cada algoritmo antes del ensemble.


## Implementacion de Metodos de Ensemble

Implemento todos los metodos de ensemble vistos en clase: Bagging, Boosting, Voting y Stacking.

In [14]:
# Funcion para implementar ensemble con bagging
def create_bagging_ensembles():
    """Crea diferentes variaciones de bagging ensemble"""
    
    bagging_ensembles = {
        'bagging_dt': BaggingClassifier(
            estimator=DecisionTreeClassifier(random_state=42),
            n_estimators=50,
            random_state=42
        ),
        'bagging_lr': BaggingClassifier(
            estimator=LogisticRegression(random_state=42, max_iter=1000),
            n_estimators=50,
            random_state=42
        ),
        'bagging_svm': BaggingClassifier(
            estimator=SVC(probability=True, random_state=42),
            n_estimators=30,  # Reducir por costo computacional
            random_state=42
        )
    }
    
    return bagging_ensembles

# Funcion para crear voting ensembles
def create_voting_ensembles():
    """Crea diferentes variaciones de voting ensemble"""
    
    # Seleccionar los mejores algoritmos base excluyendo RidgeClassifier
    best_algorithms = individual_df.head(8)['name'].tolist()
    
    # Crear estimadores para voting - excluir RidgeClassifier que no tiene predict_proba
    estimators_for_voting = []
    algorithms_need_dense = ['gaussian_nb', 'lda', 'qda']
    algorithms_no_proba = ['ridge_classifier']  # Excluir para voting soft
    
    for alg_name in best_algorithms:
        # Saltar algoritmos sin predict_proba para voting
        if alg_name in algorithms_no_proba:
            continue
            
        if alg_name in base_classifiers:
            # Solo incluir si no necesita arrays densos
            if alg_name not in algorithms_need_dense:
                estimators_for_voting.append((alg_name, base_classifiers[alg_name]))
        elif alg_name in boosting_classifiers:
            estimators_for_voting.append((alg_name, boosting_classifiers[alg_name]))
    
    # Asegurar que tengo al menos 3 estimadores
    if len(estimators_for_voting) < 3:
        # Añadir algoritmos adicionales compatibles
        backup_algorithms = ['logistic_regression', 'random_forest', 'xgboost', 'svm_linear']
        for alg_name in backup_algorithms:
            if alg_name not in [e[0] for e in estimators_for_voting]:
                if alg_name in base_classifiers and alg_name not in algorithms_no_proba:
                    estimators_for_voting.append((alg_name, base_classifiers[alg_name]))
                elif alg_name in boosting_classifiers:
                    estimators_for_voting.append((alg_name, boosting_classifiers[alg_name]))
            if len(estimators_for_voting) >= 5:
                break
    
    # Limitar a los primeros 5
    estimators_for_voting = estimators_for_voting[:5]
    
    voting_ensembles = {
        'voting_hard': VotingClassifier(
            estimators=estimators_for_voting,
            voting='hard'
        ),
        'voting_soft': VotingClassifier(
            estimators=estimators_for_voting,
            voting='soft'
        )
    }
    
    return voting_ensembles, estimators_for_voting

# Crear los ensembles
print("Creando ensembles de bagging...")
bagging_ensembles = create_bagging_ensembles()

print("Creando ensembles de voting...")
voting_ensembles, voting_estimators = create_voting_ensembles()

print(f"\nBagging ensembles creados: {len(bagging_ensembles)}")
for name in bagging_ensembles.keys():
    print(f"  - {name}")

print(f"\nVoting ensembles creados: {len(voting_ensembles)}")
for name in voting_ensembles.keys():
    print(f"  - {name}")

print(f"\nEstimadores usados en voting: {len(voting_estimators)}")
for name, _ in voting_estimators:
    print(f"  - {name}")
print("NOTA: RidgeClassifier excluido de voting por falta de predict_proba")

Creando ensembles de bagging...
Creando ensembles de voting...

Bagging ensembles creados: 3
  - bagging_dt
  - bagging_lr
  - bagging_svm

Voting ensembles creados: 2
  - voting_hard
  - voting_soft

Estimadores usados en voting: 5
  - catboost
  - extra_trees
  - logistic_regression
  - gradient_boost
  - lightgbm
NOTA: RidgeClassifier excluido de voting por falta de predict_proba


In [15]:
# Funcion para crear stacking ensembles
def create_stacking_ensembles():
    """Crea diferentes variaciones de stacking ensemble"""
    
    # Usar los mismos estimadores que en voting pero para stacking
    base_estimators = voting_estimators
    
    # Diferentes meta-learners
    stacking_ensembles = {
        'stacking_lr': StackingClassifier(
            estimators=base_estimators,
            final_estimator=LogisticRegression(random_state=42),
            cv=5,
            passthrough=False
        ),
        'stacking_rf': StackingClassifier(
            estimators=base_estimators,
            final_estimator=RandomForestClassifier(n_estimators=50, random_state=42),
            cv=5,
            passthrough=False
        ),
        'stacking_xgb': StackingClassifier(
            estimators=base_estimators,
            final_estimator=xgb.XGBClassifier(n_estimators=50, random_state=42, eval_metric='logloss'),
            cv=5,
            passthrough=False
        )
    }
    
    return stacking_ensembles

print("Creando ensembles de stacking...")
stacking_ensembles = create_stacking_ensembles()

print(f"Stacking ensembles creados: {len(stacking_ensembles)}")
for name in stacking_ensembles.keys():
    print(f"  - {name}")

# Combinar todos los ensembles
all_ensembles = {}
all_ensembles.update(bagging_ensembles)
all_ensembles.update(voting_ensembles)
all_ensembles.update(stacking_ensembles)
all_ensembles.update(boosting_classifiers)

print(f"\nTotal de ensembles a evaluar: {len(all_ensembles)}")
print("\nLista completa de ensembles:")
for i, name in enumerate(all_ensembles.keys(), 1):
    print(f"  {i:2d}. {name}")

Creando ensembles de stacking...
Stacking ensembles creados: 3
  - stacking_lr
  - stacking_rf
  - stacking_xgb

Total de ensembles a evaluar: 13

Lista completa de ensembles:
   1. bagging_dt
   2. bagging_lr
   3. bagging_svm
   4. voting_hard
   5. voting_soft
   6. stacking_lr
   7. stacking_rf
   8. stacking_xgb
   9. ada_boost
  10. gradient_boost
  11. xgboost
  12. lightgbm
  13. catboost


## Evaluacion de Todos los Ensembles




Evaluo el rendimiento de todos los metodos de ensemble implementados.

In [16]:
# SOLUCION FINAL: Evaluar ensembles usando funciones completamente aisladas
print("ENSEMBLES CON FUNCIONES AISLADAS")

def evaluar_ensemble_aislado(modelo, X_train, X_test, y_train, y_test, nombre):
    """Funcion completamente aislada para evaluar ensembles"""
    try:
        # Import local para evitar conflictos
        import sklearn.metrics
        
        # Entrenar
        modelo.fit(X_train, y_train)
        pred = modelo.predict(X_test)
        
        # Calcular metricas directamente con sklearn.metrics
        acc = sklearn.metrics.accuracy_score(y_test, pred)
        f1 = sklearn.metrics.f1_score(y_test, pred)
        
        # Probabilidades
        try:
            if hasattr(modelo, 'predict_proba'):
                prob = modelo.predict_proba(X_test)[:, 1]
                auc = sklearn.metrics.roc_auc_score(y_test, prob)
            else:
                auc = 0.0
        except:
            auc = 0.0
            
        return {
            'name': nombre,
            'accuracy': float(acc),
            'f1_score': float(f1),
            'roc_auc': float(auc),
            'success': True
        }
    except Exception as e:
        return {
            'name': nombre,
            'accuracy': 0.0,
            'f1_score': 0.0,
            'roc_auc': 0.0,
            'success': False,
            'error': str(e)
        }

ensemble_results = []

# Evaluar ensembles reales uno por uno
ensembles_a_evaluar = {
    'bagging_dt': 'Bagging',
    'bagging_lr': 'Bagging', 
    'bagging_svm': 'Bagging',
    'voting_hard': 'Voting',
    'voting_soft': 'Voting',
    'stacking_lr': 'Stacking',
    'stacking_rf': 'Stacking',
    'stacking_xgb': 'Stacking'
}

print(f"Evaluando {len(ensembles_a_evaluar)} ensembles reales...")

for nombre, tipo in ensembles_a_evaluar.items():
    if nombre in all_ensembles:
        print(f"Evaluando {nombre}...", end=" ")
        
        resultado = evaluar_ensemble_aislado(
            all_ensembles[nombre], 
            X_train_dense, 
            X_test_dense, 
            y_train, 
            y_test, 
            nombre
        )
        
        resultado['type'] = tipo
        ensemble_results.append(resultado)
        
        if resultado['success']:
            print(f"F1: {resultado['f1_score']:.4f}")
        else:
            print(f"ERROR: {resultado.get('error', 'Desconocido')[:30]}")
    else:
        print(f"Saltando {nombre} (no encontrado)")

print("\n" + "=" * 60)
exitosos = [r for r in ensemble_results if r['success']]
print(f"Ensembles exitosos: {len(exitosos)}/{len(ensembles_a_evaluar)}")

if exitosos:
    print("Resultados exitosos:")
    for r in exitosos:
        print(f"  {r['name']} ({r['type']}): F1: {r['f1_score']:.4f}")

print("Evaluacion completada.")

ENSEMBLES CON FUNCIONES AISLADAS
Evaluando 8 ensembles reales...
Evaluando bagging_dt... F1: 0.7851
Evaluando bagging_lr... F1: 0.8069
Evaluando bagging_svm... F1: 0.6994
Evaluando voting_hard... F1: 0.8417
Evaluando voting_soft... F1: 0.8465
Evaluando stacking_lr... F1: 0.8498
Evaluando stacking_rf... F1: 0.8142
Evaluando stacking_xgb... F1: 0.8161

Ensembles exitosos: 8/8
Resultados exitosos:
  bagging_dt (Bagging): F1: 0.7851
  bagging_lr (Bagging): F1: 0.8069
  bagging_svm (Bagging): F1: 0.6994
  voting_hard (Voting): F1: 0.8417
  voting_soft (Voting): F1: 0.8465
  stacking_lr (Stacking): F1: 0.8498
  stacking_rf (Stacking): F1: 0.8142
  stacking_xgb (Stacking): F1: 0.8161
Evaluacion completada.


In [17]:
# Crear DataFrame con resultados de ensemble
successful_ensemble_results = [r for r in ensemble_results if r['success']]

if len(successful_ensemble_results) > 0:
    ensemble_df = pd.DataFrame(successful_ensemble_results)
    ensemble_df = ensemble_df.sort_values('f1_score', ascending=False)

    print("Resultados de todos los ensembles (ordenados por F1-Score):")
    print(ensemble_df[['name', 'type', 'accuracy', 'f1_score', 'roc_auc']].round(4))

    # Encontrar el mejor ensemble de cada tipo
    print("\nMejor ensemble de cada tipo:")
    for ensemble_type in ensemble_df['type'].unique():
        best_of_type = ensemble_df[ensemble_df['type'] == ensemble_type].iloc[0]
        print(f"  {ensemble_type}: {best_of_type['name']} (F1: {best_of_type['f1_score']:.4f})")

    # Ensemble campeon general
    champion_ensemble = ensemble_df.iloc[0]
    print(f"\nEnsemble CAMPEON: {champion_ensemble['name']}")
    print(f"  Tipo: {champion_ensemble['type']}")
    print(f"  F1-Score: {champion_ensemble['f1_score']:.4f}")
    print(f"  Accuracy: {champion_ensemble['accuracy']:.4f}")
    print(f"  ROC-AUC: {champion_ensemble['roc_auc']:.4f}")
else:
    print("No se evaluaron ensembles exitosamente.")
    print("Esto puede deberse a problemas de compatibilidad con matrices sparse/densas.")
    print("Los algoritmos individuales funcionaron correctamente.")
    
    # Usar el mejor algoritmo individual como referencia
    champion_ensemble = individual_df.iloc[0].to_dict()
    champion_ensemble['type'] = 'Individual'
    print(f"\nUsando mejor algoritmo individual como referencia:")
    print(f"  Nombre: {champion_ensemble['name']}")
    print(f"  F1-Score: {champion_ensemble['f1_score']:.4f}")
    print(f"  Accuracy: {champion_ensemble['accuracy']:.4f}")
    print(f"  ROC-AUC: {champion_ensemble['roc_auc']:.4f}")

Resultados de todos los ensembles (ordenados por F1-Score):
           name      type  accuracy  f1_score  roc_auc
5   stacking_lr  Stacking    0.8349    0.8498   0.8903
4   voting_soft    Voting    0.8255    0.8465   0.8850
3   voting_hard    Voting    0.8208    0.8417   0.0000
7  stacking_xgb  Stacking    0.8066    0.8161   0.8885
6   stacking_rf  Stacking    0.8019    0.8142   0.8890
1    bagging_lr   Bagging    0.7877    0.8069   0.8552
0    bagging_dt   Bagging    0.7547    0.7851   0.8389
2   bagging_svm   Bagging    0.5377    0.6994   0.6677

Mejor ensemble de cada tipo:
  Stacking: stacking_lr (F1: 0.8498)
  Voting: voting_soft (F1: 0.8465)
  Bagging: bagging_lr (F1: 0.8069)

Ensemble CAMPEON: stacking_lr
  Tipo: Stacking
  F1-Score: 0.8498
  Accuracy: 0.8349
  ROC-AUC: 0.8903


In [18]:
# Visualizar la comparacion de todos los ensembles
fig = px.scatter(
    ensemble_df,
    x='f1_score',
    y='roc_auc',
    color='type',
    size='accuracy',
    hover_data=['name'],
    title="Comparacion de Todos los Metodos de Ensemble",
    labels={'f1_score': 'F1-Score', 'roc_auc': 'ROC-AUC'},
    height=500
)

# Añadir anotacion para el campeon
fig.add_annotation(
    x=champion_ensemble['f1_score'],
    y=champion_ensemble['roc_auc'],
    text=f"CAMPEON<br>{champion_ensemble['name']}",
    showarrow=True,
    arrowhead=2,
    arrowcolor="red",
    font=dict(color="red", size=12)
)

fig.show()

print("El grafico muestra la posicion de cada ensemble en terminos de F1-Score y ROC-AUC.")
print("El tamaño de los puntos representa la precision (accuracy).")

El grafico muestra la posicion de cada ensemble en terminos de F1-Score y ROC-AUC.
El tamaño de los puntos representa la precision (accuracy).


## Analisis Detallado del Mejor Ensemble

Realizo un analisis detallado del mejor ensemble encontrado, incluyendo matriz de confusion y validacion cruzada.

In [19]:
# Verificar si tenemos ensembles exitosos para analizar
if len(successful_ensemble_results) > 0 and len(ensemble_df) > 0:
    # Obtener el mejor ensemble
    best_ensemble_name = champion_ensemble['name']
    
    # Verificar que el ensemble existe en all_ensembles
    if best_ensemble_name in all_ensembles:
        best_ensemble_model = all_ensembles[best_ensemble_name]

        print(f"Analisis detallado del mejor ensemble: {best_ensemble_name}")

        # Determinar que tipo de datos usar
        if 'bagging' in best_ensemble_name or 'stacking' in best_ensemble_name or 'voting' in best_ensemble_name:
            X_train_use = X_train_dense
            X_test_use = X_test_dense
        else:
            X_train_use = X_train_tfidf
            X_test_use = X_test_tfidf

        # Entrenar nuevamente para analisis
        best_ensemble_model.fit(X_train_use, y_train)
        y_pred_best = best_ensemble_model.predict(X_test_use)
        
        if hasattr(best_ensemble_model, 'predict_proba'):
            y_pred_proba_best = best_ensemble_model.predict_proba(X_test_use)[:, 1]
        else:
            y_pred_proba_best = None

        # Reporte de clasificacion detallado
        print("\nReporte de clasificacion:")
        print(classification_report(y_test, y_pred_best, 
                                  target_names=['Fake', 'Real'], 
                                  digits=4))

        # Matriz de confusion
        cm = confusion_matrix(y_test, y_pred_best)
        print(f"\nMatriz de confusion:")
        print(cm)

        # Metricas detalladas
        tn, fp, fn, tp = cm.ravel()
        sensitivity = tp / (tp + fn)  # Recall para clase Real
        specificity = tn / (tn + fp)  # Recall para clase Fake
        precision_fake = tn / (tn + fn) if (tn + fn) > 0 else 0
        precision_real = tp / (tp + fp) if (tp + fp) > 0 else 0

        print(f"\nMetricas detalladas:")
        print(f"  Sensibilidad (Recall Real): {sensitivity:.4f}")
        print(f"  Especificidad (Recall Fake): {specificity:.4f}")
        print(f"  Precision Fake: {precision_fake:.4f}")
        print(f"  Precision Real: {precision_real:.4f}")
        print(f"  Verdaderos Positivos (Real): {tp}")
        print(f"  Verdaderos Negativos (Fake): {tn}")
        print(f"  Falsos Positivos: {fp}")
        print(f"  Falsos Negativos: {fn}")
    else:
        print(f"ERROR: El ensemble {best_ensemble_name} no existe en all_ensembles")
        print("Usando el mejor algoritmo individual para analisis:")
        
        # Usar el mejor individual como fallback
        best_individual_name = individual_df.iloc[0]['name']
        
        if best_individual_name in base_classifiers:
            best_model = base_classifiers[best_individual_name]
        elif best_individual_name in boosting_classifiers:
            best_model = boosting_classifiers[best_individual_name]
        else:
            print("No se puede encontrar el modelo para analisis")
            best_model = None
            
        if best_model:
            print(f"\nAnalizando mejor algoritmo individual: {best_individual_name}")
            
            # Determinar tipo de datos a usar
            algorithms_need_dense = ['gaussian_nb', 'lda', 'qda']
            use_dense = best_individual_name in algorithms_need_dense
            
            X_train_use = X_train_dense if use_dense else X_train_tfidf
            X_test_use = X_test_dense if use_dense else X_test_tfidf
            
            best_model.fit(X_train_use, y_train)
            y_pred_best = best_model.predict(X_test_use)
            
            cm = confusion_matrix(y_test, y_pred_best)
            print(f"Matriz de confusion (algoritmo individual):")
            print(cm)
else:
    print("No hay ensembles exitosos para analizar.")
    print("Todos los ensembles fallaron debido a problemas de compatibilidad.")
    print("Los algoritmos individuales funcionaron correctamente.")

Analisis detallado del mejor ensemble: stacking_lr

Reporte de clasificacion:
              precision    recall  f1-score   support

        Fake     0.8211    0.8125    0.8168        96
        Real     0.8462    0.8534    0.8498       116

    accuracy                         0.8349       212
   macro avg     0.8336    0.8330    0.8333       212
weighted avg     0.8348    0.8349    0.8348       212


Matriz de confusion:
[[78 18]
 [17 99]]

Metricas detalladas:
  Sensibilidad (Recall Real): 0.8534
  Especificidad (Recall Fake): 0.8125
  Precision Fake: 0.8211
  Precision Real: 0.8462
  Verdaderos Positivos (Real): 99
  Verdaderos Negativos (Fake): 78
  Falsos Positivos: 18
  Falsos Negativos: 17


In [20]:
# Visualizar la matriz de confusion del mejor ensemble
fig = px.imshow(
    cm,
    labels=dict(x="Prediccion", y="Verdadero", color="Cantidad"),
    x=['Fake', 'Real'],
    y=['Fake', 'Real'],
    color_continuous_scale='Blues',
    title=f"Matriz de Confusion - {best_ensemble_name}"
)

# Añadir texto a cada celda
for i in range(2):
    for j in range(2):
        fig.add_annotation(
            x=j, y=i,
            text=str(cm[i, j]),
            showarrow=False,
            font=dict(color="white" if cm[i, j] > cm.max()/2 else "black", size=16)
        )

fig.show()

print(f"La matriz muestra que el {best_ensemble_name} clasifica correctamente {tp + tn} de {len(y_test)} muestras.")

La matriz muestra que el stacking_lr clasifica correctamente 177 de 212 muestras.


In [21]:
# Validacion cruzada del mejor ensemble
print(f"Realizando validacion cruzada del {best_ensemble_name}...")

# Usar StratifiedKFold para mantener proporciones de clases
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Validacion cruzada con diferentes metricas
cv_scores_accuracy = cross_val_score(best_ensemble_model, X_train_tfidf, y_train, 
                                   cv=cv, scoring='accuracy', n_jobs=-1)
cv_scores_f1 = cross_val_score(best_ensemble_model, X_train_tfidf, y_train, 
                              cv=cv, scoring='f1', n_jobs=-1)
cv_scores_roc_auc = cross_val_score(best_ensemble_model, X_train_tfidf, y_train, 
                                   cv=cv, scoring='roc_auc', n_jobs=-1)

print(f"\nResultados de validacion cruzada (5-fold):")
print(f"Accuracy - Media: {cv_scores_accuracy.mean():.4f} (+/- {cv_scores_accuracy.std() * 2:.4f})")
print(f"F1-Score - Media: {cv_scores_f1.mean():.4f} (+/- {cv_scores_f1.std() * 2:.4f})")
print(f"ROC-AUC  - Media: {cv_scores_roc_auc.mean():.4f} (+/- {cv_scores_roc_auc.std() * 2:.4f})")

print(f"\nScores individuales por fold:")
print(f"Accuracy: {[f'{score:.4f}' for score in cv_scores_accuracy]}")
print(f"F1-Score: {[f'{score:.4f}' for score in cv_scores_f1]}")
print(f"ROC-AUC:  {[f'{score:.4f}' for score in cv_scores_roc_auc]}")

Realizando validacion cruzada del stacking_lr...

Resultados de validacion cruzada (5-fold):
Accuracy - Media: 0.7778 (+/- 0.0640)
F1-Score - Media: 0.8033 (+/- 0.0418)
ROC-AUC  - Media: 0.8631 (+/- 0.0387)

Scores individuales por fold:
Accuracy: ['0.7647', '0.7278', '0.7751', '0.7988', '0.8225']
F1-Score: ['0.7959', '0.7745', '0.7935', '0.8172', '0.8352']
ROC-AUC:  ['0.8636', '0.8294', '0.8630', '0.8892', '0.8704']


In [22]:
# Visualizar los resultados de validacion cruzada
cv_data = {
    'Fold': list(range(1, 6)) * 3,
    'Score': list(cv_scores_accuracy) + list(cv_scores_f1) + list(cv_scores_roc_auc),
    'Metric': ['Accuracy'] * 5 + ['F1-Score'] * 5 + ['ROC-AUC'] * 5
}

cv_df = pd.DataFrame(cv_data)

fig = px.box(
    cv_df, 
    x='Metric', 
    y='Score',
    points='all',
    title=f"Validacion Cruzada - {best_ensemble_name}",
    height=400
)

fig.show()

print("La validacion cruzada muestra la consistencia del modelo en diferentes particiones de los datos.")

La validacion cruzada muestra la consistencia del modelo en diferentes particiones de los datos.


## Meta-Ensemble de los Mejores Modelos

Creo un meta-ensemble combinando los mejores modelos de cada tipo de ensemble.

In [23]:
# DIAGNOSTICO COMPLETO: Seleccionar los mejores ensembles para meta-ensemble
print("DIAGNOSTICO: Creando meta-ensemble con los mejores modelos...")

# Verificar que tenemos ensemble_df con datos
print(f"1. Verificando ensemble_df:")
if 'ensemble_df' in locals() and len(ensemble_df) > 0:
    print(f"   - ensemble_df tiene {len(ensemble_df)} ensembles exitosos")
    print(f"   - Tipos disponibles: {ensemble_df['type'].unique()}")
    print(f"   - Mejores modelos:")
    for i, row in ensemble_df.head(3).iterrows():
        print(f"     {i+1}. {row['name']} ({row['type']}) - F1: {row['f1_score']:.4f}")
else:
    print("   - ERROR: ensemble_df esta vacio o no existe")
    print("   - No se pueden crear meta-ensembles sin ensembles exitosos")
    
    # Crear DataFrame vacio para evitar errores posteriores
    meta_df = pd.DataFrame()
    print("\n   SOLUCION: Saltando meta-ensembles")
    
    # Mostrar algoritmos individuales como referencia
    print(f"\n   ALTERNATIVA: Usando algoritmos individuales")
    print(f"   - Mejor individual: {individual_df.iloc[0]['name']} (F1: {individual_df.iloc[0]['f1_score']:.4f})")
    
# Solo si tenemos ensembles exitosos, crear meta-ensembles
print(f"\n2. Creando meta-ensembles:")
    
top_ensembles_per_type = {}
for ensemble_type in ensemble_df['type'].unique():
        best_of_type = ensemble_df[ensemble_df['type'] == ensemble_type].iloc[0]
        model_name = best_of_type['name']
        
        # Verificar que el modelo existe en all_ensembles
        if model_name in all_ensembles:
            top_ensembles_per_type[model_name] = all_ensembles[model_name]
            print(f"   - Agregado: {model_name} ({ensemble_type}) - F1: {best_of_type['f1_score']:.4f}")
        else:
            print(f"   - ERROR: {model_name} no encontrado en all_ensembles")

print(f"\n3. Verificando modelos para meta-ensemble:")
print(f"   - Modelos seleccionados: {len(top_ensembles_per_type)}")
    
if len(top_ensembles_per_type) >= 2:
        # Verificar que todos son objetos clasificador
        valid_estimators = []
        for name, model in top_ensembles_per_type.items():
            if hasattr(model, 'fit') and hasattr(model, 'predict'):
                valid_estimators.append((name, model))
                print(f"     OK {name}: {type(model).__name__}")
            else:
                print(f"     ERROR {name}: {type(model)} (NO ES CLASIFICADOR)")
        
        if len(valid_estimators) >= 2:
            print(f"\n4. Creando meta-ensembles con {len(valid_estimators)} estimadores validos...")
            
            # Crear meta-ensembles simples
            meta_ensembles = {
                'meta_voting_hard': VotingClassifier(
                    estimators=valid_estimators,
                    voting='hard'
                )
            }
            
            print(f"   - Meta-ensembles creados: {len(meta_ensembles)}")
            for name in meta_ensembles.keys():
                print(f"     - {name}")
        else:
            print(f"\n   ERROR: Solo {len(valid_estimators)} estimadores validos")
            print("   Se necesitan al menos 2 para meta-ensemble")
            meta_ensembles = {}
else:
        print(f"   ERROR: Solo {len(top_ensembles_per_type)} modelos disponibles")
        print("   Se necesitan al menos 2 para meta-ensemble")
        meta_ensembles = {}

print("\n" + "=" * 60)
print("Diagnostico de meta-ensemble completado.")

DIAGNOSTICO: Creando meta-ensemble con los mejores modelos...
1. Verificando ensemble_df:
   - ensemble_df tiene 8 ensembles exitosos
   - Tipos disponibles: ['Stacking' 'Voting' 'Bagging']
   - Mejores modelos:
     6. stacking_lr (Stacking) - F1: 0.8498
     5. voting_soft (Voting) - F1: 0.8465
     4. voting_hard (Voting) - F1: 0.8417

2. Creando meta-ensembles:
   - Agregado: stacking_lr (Stacking) - F1: 0.8498
   - Agregado: voting_soft (Voting) - F1: 0.8465
   - Agregado: bagging_lr (Bagging) - F1: 0.8069

3. Verificando modelos para meta-ensemble:
   - Modelos seleccionados: 3
     OK stacking_lr: StackingClassifier
     OK voting_soft: VotingClassifier
     OK bagging_lr: BaggingClassifier

4. Creando meta-ensembles con 3 estimadores validos...
   - Meta-ensembles creados: 1
     - meta_voting_hard

Diagnostico de meta-ensemble completado.


In [24]:
# Evaluar meta-ensembles 
print("Evaluando meta-ensembles...")

meta_results = []

# Verificar que tenemos ensembles exitosos
if len(ensemble_df) >= 2:
    print(f"Creando meta-ensembles con {len(ensemble_df)} ensembles exitosos...")
    
    # Seleccionar los TOP 3 mejores ensembles (sin importar el tipo)
    top_3_ensembles = ensemble_df.head(3)
    
    print("Ensembles seleccionados para meta-ensemble:")
    valid_estimators = []
    
    for _, row in top_3_ensembles.iterrows():
        name = row['name']
        if name in all_ensembles:
            model = all_ensembles[name]
            # Verificar que es un clasificador valido
            if hasattr(model, 'fit') and hasattr(model, 'predict'):
                valid_estimators.append((name, model))
                print(f"  OK {name} ({row['type']}) - F1: {row['f1_score']:.4f}")
            else:
                print(f"  ERROR {name} - No es un clasificador valido")
        else:
            print(f"  ERROR {name} - No encontrado en all_ensembles")
    
    if len(valid_estimators) >= 2:
        print(f"\nCreando meta-ensemble con {len(valid_estimators)} estimadores...")
        
        # Crear solo voting hard (mas compatible)
        try:
            meta_voting = VotingClassifier(
                estimators=valid_estimators,
                voting='hard'
            )
            
            print("Evaluando meta_voting_hard...", end=" ")
            
            # Usar matrices densas para maxima compatibilidad
            meta_voting.fit(X_train_dense, y_train)
            y_pred = meta_voting.predict(X_test_dense)
            
            # Calcular metricas
            from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
            
            accuracy_val = accuracy_score(y_test, y_pred)
            f1_val = f1_score(y_test, y_pred)
            
            # Para AUC, intentar obtener probabilidades
            try:
                # Voting hard no tiene predict_proba, usar score promedio de componentes
                auc_val = 0.0
                for est_name, estimator in valid_estimators:
                    if hasattr(estimator, 'predict_proba'):
                        est_proba = estimator.predict_proba(X_test_dense)[:, 1]
                        auc_val += roc_auc_score(y_test, est_proba)
                auc_val = auc_val / len(valid_estimators) if auc_val > 0 else 0.0
            except:
                auc_val = 0.0
            
            meta_results.append({
                'name': 'meta_voting_hard',
                'accuracy': accuracy_val,
                'f1_score': f1_val,
                'roc_auc': auc_val,
                'success': True
            })
            
            print(f"F1: {f1_val:.4f}, AUC: {auc_val:.4f}")
            
        except Exception as e:
            print(f"ERROR: {str(e)[:50]}...")
            meta_results.append({
                'name': 'meta_voting_hard',
                'accuracy': 0.0,
                'f1_score': 0.0,
                'roc_auc': 0.0,
                'success': False
            })
    else:
        print(f"ERROR: Solo {len(valid_estimators)} estimadores validos disponibles")
else:
    print(f"Solo {len(ensemble_df)} ensembles exitosos. Se necesitan al menos 2.")

print("\n" + "=" * 50)

# Procesar resultados
successful_meta_results = [r for r in meta_results if r['success']]

if len(successful_meta_results) > 0:
    meta_df = pd.DataFrame(successful_meta_results)
    
    print("Resultados de meta-ensembles:")
    print(meta_df[['name', 'accuracy', 'f1_score', 'roc_auc']].round(4))
    
    # Mejor meta-ensemble
    best_meta = meta_df.iloc[0]
    print(f"\nMeta-ensemble creado exitosamente:")
    print(f"  Nombre: {best_meta['name']}")
    print(f"  F1-Score: {best_meta['f1_score']:.4f}")
    print(f"  Accuracy: {best_meta['accuracy']:.4f}")
    print(f"  ROC-AUC: {best_meta['roc_auc']:.4f}")
else:
    print("No se pudieron crear meta-ensembles exitosamente.")
    print("Los ensembles individuales funcionaron bien.")
    
    # Crear DataFrame vacio para consistencia
    meta_df = pd.DataFrame()
    
    # Usar el mejor ensemble como referencia
    best_ensemble = ensemble_df.iloc[0]
    print(f"\nMejor ensemble disponible: {best_ensemble['name']}")
    print(f"  F1-Score: {best_ensemble['f1_score']:.4f}")

Evaluando meta-ensembles...
Creando meta-ensembles con 8 ensembles exitosos...
Ensembles seleccionados para meta-ensemble:
  OK stacking_lr (Stacking) - F1: 0.8498
  OK voting_soft (Voting) - F1: 0.8465
  OK voting_hard (Voting) - F1: 0.8417

Creando meta-ensemble con 3 estimadores...
Evaluando meta_voting_hard... F1: 0.8368, AUC: 0.5917

Resultados de meta-ensembles:
               name  accuracy  f1_score  roc_auc
0  meta_voting_hard     0.816    0.8368   0.5917

Meta-ensemble creado exitosamente:
  Nombre: meta_voting_hard
  F1-Score: 0.8368
  Accuracy: 0.8160
  ROC-AUC: 0.5917


## Comparacion Final de Todos los Enfoques

Comparo todos los enfoques: algoritmos individuales, ensembles simples y meta-ensembles.

In [25]:
# Combinar todos los resultados para comparacion final
print("Comparacion final de todos los enfoques:")

# Mejores de cada categoria
best_individual = individual_df.iloc[0]

# Verificar si tenemos ensemble exitoso
if len(successful_ensemble_results) > 0:
    best_ensemble = champion_ensemble
    has_ensemble = True
else:
    # Usar el mejor individual como ensemble de referencia
    best_ensemble = best_individual.to_dict()
    best_ensemble['type'] = 'Individual (fallback)'
    has_ensemble = False

comparison_summary = [
    {
        'Categoria': 'Mejor Individual',
        'Modelo': best_individual['name'],
        'F1-Score': best_individual['f1_score'],
        'ROC-AUC': best_individual['roc_auc'],
        'Accuracy': best_individual['accuracy']
    }
]

if has_ensemble:
    comparison_summary.append({
        'Categoria': 'Mejor Ensemble',
        'Modelo': best_ensemble['name'],
        'F1-Score': best_ensemble['f1_score'],
        'ROC-AUC': best_ensemble['roc_auc'],
        'Accuracy': best_ensemble['accuracy']
    })

# Verificar si tenemos meta-ensembles
if 'meta_df' in locals() and len(meta_df) > 0:
    best_meta = meta_df.iloc[0]
    comparison_summary.append({
        'Categoria': 'Mejor Meta-Ensemble',
        'Modelo': best_meta['name'],
        'F1-Score': best_meta['f1_score'],
        'ROC-AUC': best_meta['roc_auc'],
        'Accuracy': best_meta['accuracy']
    })

comparison_df = pd.DataFrame(comparison_summary)
print(comparison_df.round(4))

# Encontrar el campeon absoluto
absolute_champion = comparison_df.loc[comparison_df['F1-Score'].idxmax()]

print(f"\nCAMPEON ABSOLUTO:")
print(f"  Modelo: {absolute_champion['Modelo']}")
print(f"  Categoria: {absolute_champion['Categoria']}")
print(f"  F1-Score: {absolute_champion['F1-Score']:.4f}")
print(f"  ROC-AUC: {absolute_champion['ROC-AUC']:.4f}")
print(f"  Accuracy: {absolute_champion['Accuracy']:.4f}")

# Calcular mejora de ensemble sobre individual
if has_ensemble and best_ensemble['f1_score'] != best_individual['f1_score']:
    improvement = ((best_ensemble['f1_score'] - best_individual['f1_score']) / best_individual['f1_score']) * 100
    print(f"\nMejora de ensemble sobre algoritmo individual: {improvement:.2f}%")
else:
    print(f"\nNo hay mejora medible de ensemble (problemas de compatibilidad)")
    print(f"El mejor algoritmo individual sigue siendo la referencia")

Comparacion final de todos los enfoques:
             Categoria            Modelo  F1-Score  ROC-AUC  Accuracy
0     Mejor Individual          catboost    0.8500   0.8744    0.8302
1       Mejor Ensemble       stacking_lr    0.8498   0.8903    0.8349
2  Mejor Meta-Ensemble  meta_voting_hard    0.8368   0.5917    0.8160

CAMPEON ABSOLUTO:
  Modelo: catboost
  Categoria: Mejor Individual
  F1-Score: 0.8500
  ROC-AUC: 0.8744
  Accuracy: 0.8302

Mejora de ensemble sobre algoritmo individual: -0.03%


In [26]:
# Visualizar la comparacion final
fig = go.Figure()

# Añadir barras para cada categoria
categories = comparison_df['Categoria'].tolist()
f1_scores = comparison_df['F1-Score'].tolist()
models = comparison_df['Modelo'].tolist()

colors = ['lightblue', 'lightcoral', 'lightgreen'][:len(categories)]

fig.add_trace(go.Bar(
    x=categories,
    y=f1_scores,
    text=[f'{model}<br>F1: {score:.4f}' for model, score in zip(models, f1_scores)],
    textposition='auto',
    marker_color=colors
))

fig.update_layout(
    title="Comparacion Final: Individual vs Ensemble vs Meta-Ensemble",
    xaxis_title="Categoria",
    yaxis_title="F1-Score",
    height=500
)

# Linea horizontal para destacar al campeon
fig.add_hline(
    y=absolute_champion['F1-Score'],
    line_dash="dash",
    line_color="red",
    annotation_text=f"Campeon: {absolute_champion['F1-Score']:.4f}"
)

fig.show()

print("La visualizacion muestra la evolucion del rendimiento desde algoritmos individuales hasta meta-ensembles.")

La visualizacion muestra la evolucion del rendimiento desde algoritmos individuales hasta meta-ensembles.


## Analisis por Tipos de Ensemble

Analizo el rendimiento promedio de cada tipo de ensemble implementado.

In [27]:
# Analizar rendimiento por tipo de ensemble
print("Analisis por tipos de ensemble:")

# Verificar si tenemos ensembles exitosos
if len(successful_ensemble_results) > 0 and len(ensemble_df) > 0:
    type_analysis = ensemble_df.groupby('type').agg({
        'accuracy': ['mean', 'std', 'max'],
        'f1_score': ['mean', 'std', 'max'],
        'roc_auc': ['mean', 'std', 'max'],
        'name': 'count'
    }).round(4)

    # Renombrar columnas para claridad
    type_analysis.columns = ['_'.join(col).strip() for col in type_analysis.columns]
    type_analysis = type_analysis.rename(columns={'name_count': 'cantidad'})

    print(type_analysis)

    # Encontrar el tipo mas consistente (menor desviacion estandar)
    print("\nAnalisis de consistencia por tipo (menor std es mejor):")
    consistency_ranking = ensemble_df.groupby('type')['f1_score'].std().sort_values()
    print(consistency_ranking.round(4))

    if len(consistency_ranking) > 0:
        most_consistent_type = consistency_ranking.index[0]
        print(f"\nTipo mas consistente: {most_consistent_type} (std = {consistency_ranking.iloc[0]:.4f})")

        # Tipo con mejor rendimiento promedio
        avg_performance = ensemble_df.groupby('type')['f1_score'].mean().sort_values(ascending=False)
        best_avg_type = avg_performance.index[0]
        print(f"Tipo con mejor rendimiento promedio: {best_avg_type} (F1 = {avg_performance.iloc[0]:.4f})")
    else:
        print("No hay suficientes tipos de ensemble para analisis de consistencia")
        most_consistent_type = "N/A"
        best_avg_type = "N/A"
else:
    print("No hay ensembles exitosos para analizar por tipos.")
    print("Motivo: Todos los ensembles fallaron por problemas de compatibilidad.")
    print("\nResultados disponibles:")
    print(f"- {len(individual_df)} algoritmos individuales evaluados exitosamente")
    print(f"- Mejor individual: {individual_df.iloc[0]['name']} (F1: {individual_df.iloc[0]['f1_score']:.4f})")
    
    # Definir variables por defecto para uso posterior
    most_consistent_type = "Individual"
    best_avg_type = "Individual"

Analisis por tipos de ensemble:
          accuracy_mean  accuracy_std  accuracy_max  f1_score_mean  \
type                                                                 
Bagging          0.6934        0.1358        0.7877         0.7638   
Stacking         0.8145        0.0179        0.8349         0.8267   
Voting           0.8231        0.0033        0.8255         0.8441   

          f1_score_std  f1_score_max  roc_auc_mean  roc_auc_std  roc_auc_max  \
type                                                                           
Bagging         0.0568        0.8069        0.7873       0.1038       0.8552   
Stacking        0.0200        0.8498        0.8892       0.0009       0.8903   
Voting          0.0034        0.8465        0.4425       0.6258       0.8850   

          cantidad  
type                
Bagging          3  
Stacking         3  
Voting           2  

Analisis de consistencia por tipo (menor std es mejor):
type
Voting      0.0034
Stacking    0.0200
Bagging    

In [28]:
# Visualizar el analisis por tipos
fig = make_subplots(rows=1, cols=2, 
                    subplot_titles=['F1-Score por Tipo de Ensemble', 'Varianza por Tipo'])

# Box plot de F1-Score por tipo
for ensemble_type in ensemble_df['type'].unique():
    type_data = ensemble_df[ensemble_df['type'] == ensemble_type]
    fig.add_trace(
        go.Box(y=type_data['f1_score'], name=ensemble_type),
        row=1, col=1
    )

# Barras de desviacion estandar
fig.add_trace(
    go.Bar(
        x=consistency_ranking.index,
        y=consistency_ranking.values,
        name='Std F1-Score',
        marker_color='lightcoral'
    ),
    row=1, col=2
)

fig.update_layout(
    title="Analisis por Tipos de Ensemble",
    height=500,
    showlegend=False
)

fig.show()

print(f"El analisis muestra que {best_avg_type} tiene el mejor rendimiento promedio,")
print(f"mientras que {most_consistent_type} es el mas consistente.")

El analisis muestra que Voting tiene el mejor rendimiento promedio,
mientras que Voting es el mas consistente.


In [29]:
# Guardar todos los resultados - VERSION CORREGIDA
import os
import pickle

# Crear directorio para resultados de ensemble
if 'IN_COLAB' in globals() and IN_COLAB:
    ensemble_dir = 'ensemble_results'
else:
    ensemble_dir = '../models/ensemble_results'

os.makedirs(ensemble_dir, exist_ok=True)

print(f"Guardando resultados en: {ensemble_dir}")

# Guardar resultados de comparacion - con verificaciones
if 'individual_df' in locals() and len(individual_df) > 0:
    individual_df.to_csv(f'{ensemble_dir}/individual_algorithms_results.csv', index=False)
    print("- individual_algorithms_results.csv guardado")
else:
    print("- individual_df no existe, saltando")

if 'ensemble_df' in locals() and len(ensemble_df) > 0:
    ensemble_df.to_csv(f'{ensemble_dir}/ensemble_methods_results.csv', index=False)
    print("- ensemble_methods_results.csv guardado")
else:
    print("- ensemble_df vacío, creando archivo vacío")
    pd.DataFrame().to_csv(f'{ensemble_dir}/ensemble_methods_results.csv', index=False)

if 'meta_df' in locals() and len(meta_df) > 0:
    meta_df.to_csv(f'{ensemble_dir}/meta_ensemble_results.csv', index=False)
    print("- meta_ensemble_results.csv guardado")
else:
    print("- meta_df vacío, creando archivo vacío")
    pd.DataFrame().to_csv(f'{ensemble_dir}/meta_ensemble_results.csv', index=False)

if 'comparison_df' in locals():
    comparison_df.to_csv(f'{ensemble_dir}/final_comparison.csv', index=False)
    print("- final_comparison.csv guardado")
else:
    print("- comparison_df no existe, saltando")

# Guardar analisis por tipos - con verificaciones
if 'type_analysis' in locals():
    type_analysis.to_csv(f'{ensemble_dir}/ensemble_types_analysis.csv')
    print("- ensemble_types_analysis.csv guardado")

if 'consistency_ranking' in locals():
    consistency_ranking.to_csv(f'{ensemble_dir}/consistency_ranking.csv')
    print("- consistency_ranking.csv guardado")

if 'avg_performance' in locals():
    avg_performance.to_csv(f'{ensemble_dir}/average_performance_by_type.csv')
    print("- average_performance_by_type.csv guardado")

# Guardar configuracion experimental - con verificaciones de variables
experiment_config = {
    'dataset_size': len(df_clean) if 'df_clean' in locals() else 0,
    'train_size': len(y_train) if 'y_train' in locals() else 0,  # Usar y_train que sabemos que existe
    'test_size': len(y_test) if 'y_test' in locals() else 0,     # Usar y_test que sabemos que existe
    'tfidf_features': X_train_tfidf.shape[1] if 'X_train_tfidf' in locals() and hasattr(X_train_tfidf, 'shape') else 0,
    'algorithms_evaluated': len(individual_df) if 'individual_df' in locals() else 0,
    'ensembles_evaluated': len(ensemble_df) if 'ensemble_df' in locals() else 0,
    'champion_model': absolute_champion['Modelo'] if 'absolute_champion' in locals() else 'CatBoost',
    'champion_f1': float(absolute_champion['F1-Score']) if 'absolute_champion' in locals() else 0.85,
    'improvement_over_individual': float(improvement) if 'improvement' in locals() else 0.0
}

with open(f'{ensemble_dir}/experiment_config.pickle', 'wb') as f:
    pickle.dump(experiment_config, f)
print("- experiment_config.pickle guardado")

# Guardar otros objetos si existen
if 'tfidf_vectorizer' in locals():
    with open(f'{ensemble_dir}/tfidf_vectorizer.pickle', 'wb') as f:
        pickle.dump(tfidf_vectorizer, f)
    print("- tfidf_vectorizer.pickle guardado")

if 'label_encoder' in locals():
    with open(f'{ensemble_dir}/label_encoder.pickle', 'wb') as f:
        pickle.dump(label_encoder, f)
    print("- label_encoder.pickle guardado")

print("\nGuardado de resultados completado exitosamente.")
print(f"Todos los archivos están en: {ensemble_dir}")

Guardando resultados en: ../models/ensemble_results
- individual_algorithms_results.csv guardado
- ensemble_methods_results.csv guardado
- meta_ensemble_results.csv guardado
- final_comparison.csv guardado
- ensemble_types_analysis.csv guardado
- consistency_ranking.csv guardado
- average_performance_by_type.csv guardado
- experiment_config.pickle guardado
- tfidf_vectorizer.pickle guardado
- label_encoder.pickle guardado

Guardado de resultados completado exitosamente.
Todos los archivos están en: ../models/ensemble_results
